In [12]:
import sys
import os
src_path = os.path.abspath(os.path.join(os.getcwd(), "src"))
sys.path.append(src_path)

from ocr.base_ocr import BaseOCRUtil

ImportError: cannot import name 'BaseOCRUtil' from 'ocr.base_ocr' (c:\Users\sridh\OneDrive\Desktop\webdev\CrossOCR\src\ocr\base_ocr.py)